In [1]:
# AbacusResearch/RasGulla1-7b

In [1]:
# !pip install -qU transformers datasets loralib einops xformers

In [2]:
# !pip install git+https://github.com/huggingface/accelerate.git

In [3]:
# !pip install -qqq torch

In [4]:
# !pip install -i https://pypi.org/simple/ bitsandbytes

In [5]:
# !pip install peft

Installing torch and unsloth had been a nightmare since the torch version on colab breaks unsloth for some reason, following code taken from unsloth's huggingface account https://huggingface.co/datasets/unsloth/notebooks/blob/main/DPO_Zephyr_Unsloth_Example.ipynb

In [1]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

# DO NOT RUN, TO CHECK THE PREVIOUS ISSUE WITH TORCH

In [5]:
# this is old
import torch
print(torch.__version__)

2.1.0+cu121


# After unsloth's notebook torch, run from here

In [2]:
print(torch.__version__)

2.2.1+cu121


In [3]:
major_version, minor_version = torch.cuda.get_device_capability()
print(f"CUDA major version: {major_version}, minor version: {minor_version}")

CUDA major version: 7, minor version: 0


In [4]:
from unsloth import FastLanguageModel

# Using transformers, will switch to unsloth

In [20]:
# quantization
# quiantization_config = BitsAndBytesConfig(load_in_4bit=True)

In [21]:
# tokenizer
# tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b")

In [ ]:
# load model with 4bit quantization with the config
# model = AutoModelForCausalLM.from_pretrained("google/gemma-7b", quantization_config=quiantization_config)

# unsloth

In [5]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "google/gemma-7b",
    max_seq_length = 4096,
    dtype = None,
    load_in_4bit = True,
)

Unsloth: You passed in `google/gemma-7b` and `load_in_4bit = True`.
We shall load `unsloth/gemma-7b-bnb-4bit` for 4x faster loading.


config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Gemma patching release 2024.3
   \\   /|    GPU: Tesla V100-SXM2-16GB. Max memory: 15.773 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.57G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # 0 is optimized
    bias = "none",    # "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # rank stabilized
    loftq_config = None
)

Unsloth 2024.3 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [86]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="sha256_password.csv")

In [7]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="div_move_exp_5000.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'prev_moves', 'last_move_fill', 'explanation'],
        num_rows: 5000
    })
})

In [46]:
prompt = """
### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

In [59]:
def formatting_prompts_func(examples):
    instruction = "below is a sha256 representation of a password, please generate the password -"
    inputs       = examples["sha_256"]
    outputs      = examples["password"]
    texts = []
    for input, output in zip(inputs, outputs):
        text = prompt.format(instruction, input, output) + EOS_TOKEN
        #print(text)
        texts.append(text)
    return { "text" : texts, }

In [19]:
prompt_chess = """
Instruction:{}; previous moves:{}; last move:{}. Response:{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

In [20]:
def formatting_prompts_func_chess(examples):
    instruction = "in a paragraph, explain the rationale behind the last move, where all previous moves are:"
    prevs       = examples["prev_moves"]
    lasts       = examples["last_move_fill"]
    exps     = examples["explanation"]
    texts = []
    for prev, last, exp in zip(prevs, lasts, exps):
        text = prompt_chess.format(instruction, prev, last, exp) + EOS_TOKEN
        #print(text)
        texts.append(text)
    return { "text" : texts, }

In [21]:
dataset = dataset.map(formatting_prompts_func_chess, batched = True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [12]:
dataset["train"][1]

{'Unnamed: 0.1': 1,
 'Unnamed: 0': 1,
 'prev_moves': 'd4',
 'last_move_fill': 'd5',
 'explanation': "The move d5 by black is a common response to white's move d4 in chess. By playing d5, black aims to control the center of the board and challenge white's dominance in that area. This move also opens up lines for the black queen and bishop to potentially develop. Additionally, by advancing the pawn to d5, black prepares to potentially push it further to d4 to create pressure on white's pieces and limit their mobility. Overall, playing d5 is a strategic move that aims to equalize the game and create additional options for black's pieces to develop.",
 'text': " \n### Instruction:\nin a paragraph, explain the rationale behind the last move, where all previous moves are:\n### previous moves:\nd4\n### last move:\nd5\n### Response:\nThe move d5 by black is a common response to white's move d4 in chess. By playing d5, black aims to control the center of the board and challenge white's dominanc

In [13]:
from trl import SFTTrainer
from transformers import TrainingArguments

In [22]:
dataset["train"]["text"][0]

" \nInstruction:in a paragraph, explain the rationale behind the last move, where all previous moves are:; previous moves:d4; last move:d4. Response:The last move of playing d4 again likely aims to continue controlling the center of the board and establishing a strong pawn presence. By playing d4 for a second time, the player may be reinforcing their control over the central squares and preparing to develop their pieces for a more aggressive and strategic play. Additionally, repeating the move d4 could potentially be a part of a specific opening or strategy that the player is following, such as the Queen's Gambit or the King's Indian Defense. Overall, playing d4 again maintains a solid foundation for future moves and sets the stage for a dynamic and flexible game plan.<eos>"

In [23]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    dataset_text_field = "text",
    max_seq_length = 4096,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # max_steps = 1000,
        num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/5000 [00:00<?, ? examples/s]

In [24]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 625
 "-____-"     Number of trainable parameters = 50,003,968


Step,Training Loss
1,2.168700
2,2.258200
3,2.066000
4,1.877400
5,1.661600
6,1.575800
7,1.478700
8,1.307500
9,1.256500
10,1.197600


In [85]:
trainer_stats

TrainOutput(global_step=60, training_loss=2.1221025546391803, metrics={'train_runtime': 220.354, 'train_samples_per_second': 2.178, 'train_steps_per_second': 0.272, 'total_flos': 2395036607090688.0, 'train_loss': 2.1221025546391803, 'epoch': 0.05})

In [72]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    prompt.format(
        "below is a sha256 representation of a password, please generate the password -",
        "4007d46292298e83da10d0763d95d5139fe0c157148d0587aa912170414ccba6", # input, AstonMartinDB11
        "", # output - expected response
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<bos> 
### Instruction:
below is a sha256 representation of a password, please generate the password -

### Input:
4007d46292298e83da10d0763d95d5139fe0c157148d0587aa912170414ccba6

### Response:
123456789<eos>


In [26]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    prompt_chess.format(
        "in a paragraph, explain the rationale behind the last move, where all previous moves are",
        "e4 c5 Nf3 Nc6 Bc4 g6 Ng5 Ne5 Bb3 h6 Nf3 Bg7 Nxe5 Bxe5 O-O Qc7 Qf3 Nf6 h3 h5 c3 O-O d3 Nh7 g4 hxg4 Qxg4 d6 Qxg6+ Kh8 Bxf7 Bxh3 Re1 Rg8 Bxg8 Rxg8 Qxg8+ Kxg8 Re3",
        "Qc8",
        ""
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<bos> 
Instruction:in a paragraph, explain the rationale behind the last move, where all previous moves are; previous moves:e4 c5 Nf3 Nc6 Bc4 g6 Ng5 Ne5 Bb3 h6 Nf3 Bg7 Nxe5 Bxe5 O-O Qc7 Qf3 Nf6 h3 h5 c3 O-O d3 Nh7 g4 hxg4 Qxg4 d6 Qxg6+ Kh8 Bxf7 Bxh3 Re1 Rg8 Bxg8 Rxg8 Qxg8+ Kxg8 Re3; last move:Qc8. Response:The last move, Qc8, was likely made to protect the bishop on h3 and prevent it from being captured by the opponent's queen. By moving the queen to c8, the player is also preparing to potentially launch an attack on the opponent's king side. Additionally, the queen on c8 is now in a more centralized position, allowing for better control of the board and potential threats to the opponent's pieces. Overall, the move Qc8 was a strategic decision to improve the player's position and maintain control of the game.<eos>
